In [7]:
# Imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pickle

# For functions pertaining to the operating system
import os

# Module for working with URLs
import urllib

# For performing operations on zip files
import zipfile

# Makes it possible to read and write tar archives including gzip, bz2 and lzma compression
import tarfile

# Module for working with URLs
from six.moves import urllib

# Deep Learning
import tensorflow as tf
from tensorflow.contrib.training import HParams
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization, AveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Visualization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [6]:
# Define hyperparameters
params = HParams(
    
    n_classes=10, # Number of classes in the Cifar 10 dataset
    learning_rate=1e-4, # How much to change the model in response to the estimated error each time the weights are updated
    train_batch_size=32, # Training batch size; number of data points in one forward/backward pass
    val_batch_size=32, # Batch size for validation; number of data points in one forward/backward pass
    test_batch_size=32, # Testing batch size; number of data points in one forward/backward pass
    n_epochs=10, # One epoch is equivalent to one forward and one backward pass of all training data points
    input_name='input_one', # Name of NN input layer
    data_dir='/tmp/cifar-data/' # Path to data

)

AttributeError: module 'tensorflow' has no attribute 'HParams'

In [0]:
# Dimension of the square image
n_pixels = 32

# Number of image color channels
n_channels = 3

# Length of flattened image
size_flat = n_channels * (n_pixels**2)

# Number of classes considered
n_classes = params.n_classes

# Quantity of files in the original dataset
_n_files_train = 1

# Quantity of training images
_n_images_train = 60000

In [0]:
def extract_data(destination):
    """
    Description - Extracts the data into the directory specified by the params 
                  class.

    Returns - N/A
    """

    # Set filename
    filename = 'cifar-100-python.tar.gz'

    # Create the final file path
    destination_path = os.path.join(destination, filename)

    # If the file does not exist, 
    if not os.path.exists(destination_path):

        # If the directory does not exist,
        if not os.path.exists(destination):

            # Create the needed directory 
            os.makedirs(destination)

        # Download the dataset using urllib
        destination_path, _ = urllib.request.urlretrieve(url=url, filename=destination_path)

        # Print a status message
        print()
        print("Step (1/2) - Files have been downloaded.")

        # If we are dealing with a zip file,
        if destination_path.endswith(".zip"):

            # extract the zipfile into the desitnation directory
            zipfile.ZipFile(file=destination_path, mode="r").extractall(destination)

        # If we are dealing with a tar file, extract with tarfile
        elif destination_path.endswith((".tar.gz", ".tgz")):

            # Extract the tarfile into the desitnation directory
            tarfile.open(name=destination_path, mode="r:gz").extractall(destination)

            # Print a status message
            print("Step (2/2) - Files have been extracted.")

    # If the data does exist, 
    else:

        # Print a status message
        print("Data has apparently already been saved locally and extracted.")

In [0]:
def _get_file_path(filename=""):
    """
    Description - Finds and returns the data path location.
    """

    # Use the join method to create the path
    return os.path.join(params.data_dir, "cifar-100-python/", filename)

In [0]:
def _unpickle(filename):
    """
    Description - Unpickle (de-serialize) the given file pieces and return the 
                  aggregated data chunk.
    """

    # Get the file path using the function defined above
    destination_path = _get_file_path(filename)

    # Print a status message
    print("Currently loading data from: " + destination_path)

    # Open the file located at file_path
    with open(destination_path, mode='rb') as file:

        # Load the data into a new variable using Pickle's load method
        data = pickle.load(file, encoding='bytes')

    return data

In [0]:
def _convert_images(raw_data):
    """
    Description - Preprocesses raw image data and convert to a 4-dimensional 
    array: [image_number, height, width, channel]
   
    Returns - The preprocessed and scaled image data.
    """

    # Scale the pixel data
    scaled_data = np.array(raw_data, dtype=float) / 255.0

    # Change the shape of the array to 4-D
    images = scaled_data.reshape([-1, n_channels, img_size, img_size])

    # Reindex the array
    images = images.transpose([0, 2, 3, 1])

    return images

In [0]:
def _load_data(filename):
    """
    Description - Unpickles (de-serializes) the input file and converts the data
                  to the data shape specified in the _convert_data function.
    
    Returns - The converted data and the class label for each image.
    """

    # Unpickle the data
    data = _unpickle(filename)

    # Retrieve the raw image pixel data
    raw_images = data[b'data']

    # Arrange the class labels into a numpy array
    classes = np.array(data[b'labels'])

    # Convert the image pixel size/orientation
    images = _convert_images(raw_images)

    return images, classes

In [0]:
def load_class_names():

    # Unpickle the file and access the class label names
    raw_classes = _unpickle(filename="meta")[b'label_names']

    # Convert from strings to a list
    names = [x.decode('utf-8') for x in raw]

    return names

In [0]:
def load_training_data():
    """
    Description - Builds numpy arrays containing the image data and the class 
                  labels from the 5 files in the Cifar 100 dataset.

    Returns - The images and class labels for each training image data point 
              in the set.
    """

    # Allocate memory for the images
    images = np.zeros(shape=[_n_images_train, n_pixels, n_pixels, n_channels], dtype=float)

    # Allocate memory for the class labels
    classes = np.zeros(shape=[_n_images_train], dtype=int)

    # Load the images and class labels from the training data
    images, classes = _load_data(filename="train")

    return images, classes

In [0]:
def load_validation_data():
    """
    Description - Loads 5000 data points from the test batch file of the Cifar
                  100 dataset to be used for validation.

    Returns - The images and class labels for each validation image data point 
              in the set.
    """

    # Load the images and class labels from the test batch file
    images, classes = _load_data(filename="test")

    # Define the validation data to be all but the first 5000 data points
    images = images[5000:, :, :, :]
    classes = classes[5000:]

    return images, classes

In [0]:
def load_testing_data():
    """
    Description - Loads 5000 data points from the test batch file of the Cifar
                  100 dataset to be used for testing.

    Returns - The images and class labels for each testing image data point 
              in the set.
    """

    # Load the unpickled and converted data
    images, classes = _load_data(filename="test")

    # Define the testing data to be the first 5000 data points
    images = images[:5000, :, :, :]
    classes = classes[:5000]

    return images, classes

In [0]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs - This is the 4-D tensor input from the image preprocessing step
        num_filters - The number of image convolution filters
        kernel_size - Dimensions of the 2D convolution window
        strides - Specifies the number of strides along the height and width
        activation - Neural network activation function
        batch_normalization - Normalize the activations at each layer
        conv_first - Use true to indicate conv-bn-activation; use false
                     to indicate bn-activation-conv
            
    # Returns
        x - Tensor that is used as an input to the next layer
    """

    # Declare a 2 dimensional convolutional layer; params above
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=tf.keras.regularizers.l2(1e-4))

    # X is the number of inputs
    x = inputs

    # If conv_first is True, apply conv to x
    if conv_first:
        x = conv(x)
        # If batch_normalization is True, apply BN to x
        if batch_normalization:
            x = BatchNormalization()(x)
        # If activation has some value, apply activation to x
        if activation is not None:
            x = Activation(activation)(x)
    # Else, apply conv to x after applying BN and activation
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v2(input_shape, batch_size=1024, depth=20, num_classes=10):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:

    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape - Dimensions of the input image tensor
        depth - Number of convolutional layers
        num_classes - Total number of class labels
    # Returns
        model - A Keras model
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
      

    # Number of input filters
    num_filters_in = 16

    # Number of residual blocks
    num_res_blocks = int((depth - 2) / 9)

    # Define input layer with given input shape and batch size
    inputs = Input(shape=input_shape, batch_size=batch_size)
                   
    
    # Create a residual layer
    x = resnet_layer(inputs=inputs, num_filters=num_filters_in, conv_first=True)
                     
    # Create three stages
    for stage in range(3):

        # Create a number of residual blocks
        for res_block in range(num_res_blocks):

            # Use the rectified linear unit
            activation = 'relu'

            # Set mean to zero and standard deviation to 1
            batch_normalization = True

            # This will set the number of strides across the image to 1
            strides = 1

            # At the first stage
            if stage == 0:

                # Set the number of output filters
                num_filters_out = num_filters_in * 4

                # This is the first layer in the first stage
                if res_block == 0:  
                    activation = None
                    batch_normalization = False

            # At all other stages
            else:

                # Set the number of output filters
                num_filters_out = num_filters_in * 2

                # At the first layer
                if res_block == 0: 

                    # This is amount by which the filter shifts across the image
                    strides = 2    

            # These are the residual layers
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            
            # At the first layer
            if res_block == 0:

                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
                
            x = tf.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)

    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=params.learning_rate),
                  metrics=['accuracy'])

    return model

In [0]:
# # Set the verbosity of TensorFlow
# tf.logging.set_verbosity(tf.logging.DEBUG)

# # Define the residual net layer using resnet v2; borrowed from the official Keras examples
# model = resnet_v2((32, 32, 3), depth=56, num_classes=params.n_classes)

# # Set model variables
# X_train, y_train = load_training_data()

# # Set validation variables
# X_val, y_val = load_validation_data()

# # Set model testing variables
# X_test, y_test = load_testing_data()

# # This will enable use of the TensorBoard for visualization
# callbacks = [ tf.keras.callbacks.TensorBoard(log_dir=params.checkpoint_dir) ]

# # Preprocess and augment the data
# # (Fill in argument comments)
# augmented = ImageDataGenerator(
    
#     featurewise_center=False, # 
#     samplewise_center=False, #
#     featurewise_std_normalization=False, #
#     samplewise_std_normalization=False, #
#     zca_whitening=False, #
#     zca_epsilon=1e-6, #
#     rotation_range=0, #
#     width_shift_range=0.1, #
#     height_shift_range=0.1, #
#     fill_mode='nearest', #
#     cval=0., #
#     horizontal_flip=True, #
#     vertical_flip=False #

# )

# # Generates preprocessed and augmented data for the purpose of improving
# # the models ability to generalize to new data
# augmented.fit(X_train)

# # Fit the model; use fit_generator to account for the generator used to augment
# # and preprocess the data
# model.fit_generator(
    
#     augmented.flow(X_train, y_train, batch_size=params.train_batch_size),
#     epochs=params.n_epochs,
#     validation_data=(X_val, y_val),
#     workers=4,
#     callbacks=callbacks

#                     )

In [248]:
data = _unpickle('train')

Currently loading data from: /tmp/cifar-data/cifar-100-python/train


FileNotFoundError: ignored